In [5]:
import pandas as pd
import numpy as np

In [6]:
%%time
import funcs

rats_gdo = funcs.get_rats_gdo()
rats_gdo.head(3)

CPU times: user 789 ms, sys: 4.47 ms, total: 794 ms
Wall time: 870 ms


,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,NOM_OPERACAO,...,CIA,VIATURAS,EFETIVO,EFETIVIDADE_PARCIAL,Qde de locais fiscalizados,Qde de pessoas abordadas,Qde de pessoas que sopraram o etilometro,Qde de veiculos fiscalizados,EFICIENCIA_E_EFICACIA,EFETIVIDADE
RAT.NUM_ATIVIDADE,,,,,,,,,,,,,,,,,,,,,
2019-006234146-001,Y07001,OPERACAO DE BATIDA POLICIAL,09/02/2019,09/02/2019,14:19,09/02/2019,23:30,VIA DE ACESSO PUBLICA,VIA DE ACESSO PUBLICA,,...,53 CIA,1,2,0,0,0,0,0,0,0
2019-007702215-001,Y07001,OPERACAO DE BATIDA POLICIAL,18/02/2019,18/02/2019,14:04,18/02/2019,19:00,PROPRIETARIO / FUNCIONARIO DE ESTABELECIMENTO,VIA DE ACESSO PUBLICA,,...,53 CIA,1,1,0,0,0,0,0,0,0
2019-008142426-001,Y07001,OPERACAO DE BATIDA POLICIAL,20/02/2019,20/02/2019,14:14,20/02/2019,19:00,PROPRIETARIO / FUNCIONARIO DE ESTABELECIMENTO,VIA DE ACESSO PUBLICA,,...,53 CIA,1,1,0,0,0,0,0,0,0


<h2 style='color:#009'>Código para gerar relatório de LEI SECA, por Cias, por ano, por mês</h2>

In [3]:
rats_ols = rats_gdo[        
    (~rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('PRV'))
    & (rats_gdo['NAT.CODIGO'] == 'Y04012')
]
rats_ols.loc[:,'CIA_OLS'] = np.select([
    rats_ols.loc[:,'NOM_UNID_RESPONSAVEL'].str.contains('240')
], [
    '240 CIA'
    ], default = rats_ols.loc[:,'CIA'])

ols_by_cia = rats_ols[
    rats_ols['ANO'] >= 2020
].groupby(['ANO','MES','CIA_OLS']).sum()['EFETIVIDADE'].unstack(['MES']).fillna(0)

ols_by_cia.loc[:,'ACUM'] = ols_by_cia.sum(1)
ols_by_cia.loc[(2020, '23 BPM'),:] = ols_by_cia.sum()

# rats_ols

ols_by_cia = ols_by_cia.astype('int16')
ols_by_cia.columns =  pd.MultiIndex.from_product([['OLS'], ols_by_cia.columns])
ols_by_cia

/home/p323bpm/.asdf/installs/python/anaconda3-2020.02/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/p323bpm/.asdf/installs/python/anaconda3-2020.02/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


OLS                       
MES            1  2  3   4   5   6 ACUM
ANO  CIA_OLS                           
2020 139 CIA   1  0  1   2   0   2    6
     142 CIA   0  1  0   0   4   2    7
     240 CIA   0  1  2   7   6   6   22
     51 CIA    0  3  1   2   3   3   12
     53 CIA    2  0  0   1   0   0    3
     23 BPM    3  5  4  12  13  13   50

<h2 style='color:#009'>Código para gerar relatório de RQV, por setores, por ano, por mês</h2>

In [7]:
rats_rqv = rats_gdo[
    (rats_gdo['NAT.CODIGO'].str.contains('Y0700[1345]')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('IND PE')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('BPE')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('CPE')) &
    (rats_gdo['ANO'] == 2020)
]

rqv_by_cia = rats_rqv.groupby(['MES','CIA']).sum()[['EFICIENCIA_E_EFICACIA','EFETIVIDADE']]
rqv_by_cia = rqv_by_cia.unstack('MES')
rqv_by_cia.loc['23 BPM'] = rqv_by_cia.sum()
rqv_by_cia.loc[:,('EFICIENCIA_E_EFICACIA', 'ACUM')] = rqv_by_cia.loc[:,'EFICIENCIA_E_EFICACIA'].sum(axis=1)
rqv_by_cia.loc[:,('EFETIVIDADE', 'ACUM')] = rqv_by_cia.loc[:,'EFETIVIDADE'].sum(axis=1)
rqv_by_cia = rqv_by_cia[['EFICIENCIA_E_EFICACIA', 'EFETIVIDADE']]
rqv_by_cia.fillna(0)

EFICIENCIA_E_EFICACIA                                 EFETIVIDADE  \
MES                         1     2    3    4    5    6  ACUM           1   
CIA                                                                         
139 CIA                   238   270  232  202  245  243  1430          45   
142 CIA                   454   399  361  233  210  185  1842         147   
51 CIA                    273   163  135  127  193  184  1075         109   
53 CIA                    249   198  213  187  225  167  1239          55   
23 BPM                   1214  1030  941  749  873  779  5586         356   

                                        
MES        2    3    4    5    6  ACUM  
CIA                                     
139 CIA   46   41   44   59   39   274  
142 CIA  147  141   90   72   54   651  
51 CIA    63   70   56   99   90   487  
53 CIA    30   28   23   24   18   178  
23 BPM   286  280  213  254  201  1590

In [8]:
writer = pd.ExcelWriter('rqv_e_ols.xlsx', engine='xlsxwriter')
rqv_by_cia.to_excel(writer, sheet_name='RQV')
ols_by_cia.to_excel(writer, sheet_name='OLS')
rats_gdo.to_excel(writer, sheet_name='Todos os registros')
writer.save()